<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/czyzewska/Systemy_rekomendacyjne_14_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mixtend
import pandas as pd

ERROR: Could not find a version that satisfies the requirement mixtend (from versions: none)
ERROR: No matching distribution found for mixtend


In [20]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
df = pd.read_csv('GroceryStoreDataSet.csv')
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,"MILK,BREAD,BISCUIT"
0,"BREAD,MILK,BISCUIT,CORNFLAKES"
1,"BREAD,TEA,BOURNVITA"
2,"JAM,MAGGI,BREAD,MILK"
3,"MAGGI,TEA,BISCUIT"
4,"BREAD,TEA,BOURNVITA"
5,"MAGGI,TEA,CORNFLAKES"
6,"MAGGI,BREAD,TEA,BISCUIT"
7,"JAM,MAGGI,BREAD,TEA"
8,"BREAD,MILK"
9,"COFFEE,COCK,BISCUIT,CORNFLAKES"


In [27]:
def correct_df_items(df):
    corrections = {'COCK': 'CHEESE', 'SUGER': 'SUGAR'}
    df['products'] = df['products'].replace(corrections, regex=True)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
df = pd.read_csv('GroceryStoreDataSet.csv', names = ['products'], sep = ',')
df = correct_df_items(df)
print(df.shape)
df.head(5)

(20, 1)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,products
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


In [29]:
df.value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


products                        
BREAD,TEA,BOURNVITA                 2
COFFEE,CHEESE,BISCUIT,CORNFLAKES    2
BREAD,COFFEE,SUGAR                  2
BREAD,COFFEE,CHEESE                 1
JAM,MAGGI,BREAD,MILK                1
MILK,BREAD,BISCUIT                  1
MAGGI,TEA,CORNFLAKES                1
MAGGI,TEA,BISCUIT                   1
MAGGI,BREAD,TEA,BISCUIT             1
JAM,MAGGI,BREAD,TEA                 1
COFFEE,SUGAR,BOURNVITA              1
COFFEE,SUGAR,CORNFLAKES             1
BREAD,SUGAR,BOURNVITA               1
BREAD,SUGAR,BISCUIT                 1
BREAD,MILK,BISCUIT,CORNFLAKES       1
BREAD,MILK                          1
TEA,MILK,COFFEE,CORNFLAKES          1
Name: count, dtype: int64

In [30]:
data = list(df['products'].apply(lambda x:x.split(',')))
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'CHEESE', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'CHEESE', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGAR', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'CHEESE'],
 ['BREAD', 'SUGAR', 'BISCUIT'],
 ['COFFEE', 'SUGAR', 'CORNFLAKES'],
 ['BREAD', 'SUGAR', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGAR'],
 ['BREAD', 'COFFEE', 'SUGAR'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [31]:
te = TransactionEncoder()

data_encoded = te.fit(data).transform(data)
df_bool = pd.DataFrame(data_encoded, columns=te.columns_)
df_bool.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BISCUIT,BOURNVITA,BREAD,CHEESE,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGAR,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True


In [35]:
df_encoded = df_bool.astype(int)
df_encoded

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BISCUIT,BOURNVITA,BREAD,CHEESE,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGAR,TEA
0,1,0,1,0,0,0,0,0,1,0,0
1,1,0,1,0,0,1,0,0,1,0,0
2,0,1,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,1,1,1,0,0
4,1,0,0,0,0,0,0,1,0,0,1
5,0,1,1,0,0,0,0,0,0,0,1
6,0,0,0,0,0,1,0,1,0,0,1
7,1,0,1,0,0,0,0,1,0,0,1
8,0,0,1,0,0,0,1,1,0,0,1
9,0,0,1,0,0,0,0,0,1,0,0


In [40]:
from mlxtend.frequent_patterns import apriori

support = apriori(df_bool, min_support=0.15, use_colnames=True).sort_values(by='support', ascending = False)
support
#najczęściej kupowany chlep 65%, najczęstsze połączenie mleko i chleb 20%

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
2,0.65,(BREAD)
4,0.40,(COFFEE)
0,0.35,(BISCUIT)
9,0.35,(TEA)
5,0.30,(CORNFLAKES)
8,0.30,(SUGAR)
6,0.25,(MAGGI)
7,0.25,(MILK)
15,0.20,"(MILK, BREAD)"
20,0.20,"(SUGAR, COFFEE)"


In [51]:
conf = association_rules(support, metric='zhangs_metric', min_threshold=0.15).sort_values(by='zhangs_metric', ascending=False)
conf

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(COFFEE),(CHEESE),0.40,0.15,0.15,0.375000,2.500000,0.0900,1.360000,1.000000
7,(TEA),(MAGGI),0.35,0.25,0.20,0.571429,2.285714,0.1125,1.750000,0.865385
6,(MAGGI),(TEA),0.25,0.35,0.20,0.800000,2.285714,0.1125,3.250000,0.750000
11,(CHEESE),(COFFEE),0.15,0.40,0.15,1.000000,2.500000,0.0900,inf,0.705882
3,(COFFEE),(SUGAR),0.40,0.30,0.20,0.500000,1.666667,0.0800,1.400000,0.666667
4,(COFFEE),(CORNFLAKES),0.40,0.30,0.20,0.500000,1.666667,0.0800,1.400000,0.666667
2,(SUGAR),(COFFEE),0.30,0.40,0.20,0.666667,1.666667,0.0800,1.800000,0.571429
5,(CORNFLAKES),(COFFEE),0.30,0.40,0.20,0.666667,1.666667,0.0800,1.800000,0.571429
1,(BREAD),(MILK),0.65,0.25,0.20,0.307692,1.230769,0.0375,1.083333,0.535714
12,(BISCUIT),(CORNFLAKES),0.35,0.30,0.15,0.428571,1.428571,0.0450,1.225000,0.461538


In [49]:
conf_2 = association_rules(support, metric="confidence",min_threshold=0.15).sort_values(by='zhangs_metric')
conf_2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
17,(BREAD),(COFFEE),0.65,0.40,0.15,0.230769,0.576923,-0.1100,0.780000,-0.676923
16,(COFFEE),(BREAD),0.40,0.65,0.15,0.375000,0.576923,-0.1100,0.560000,-0.550000
7,(BREAD),(TEA),0.65,0.35,0.20,0.307692,0.879121,-0.0275,0.938889,-0.282051
13,(BREAD),(BISCUIT),0.65,0.35,0.20,0.307692,0.879121,-0.0275,0.938889,-0.282051
15,(BREAD),(MAGGI),0.65,0.25,0.15,0.230769,0.923077,-0.0125,0.975000,-0.192308
6,(TEA),(BREAD),0.35,0.65,0.20,0.571429,0.879121,-0.0275,0.816667,-0.174603
12,(BISCUIT),(BREAD),0.35,0.65,0.20,0.571429,0.879121,-0.0275,0.816667,-0.174603
14,(MAGGI),(BREAD),0.25,0.65,0.15,0.600000,0.923077,-0.0125,0.875000,-0.100000
8,(SUGAR),(BREAD),0.30,0.65,0.20,0.666667,1.025641,0.0050,1.050000,0.035714
9,(BREAD),(SUGAR),0.65,0.30,0.20,0.307692,1.025641,0.0050,1.011111,0.071429


### Rekomendacja Filmów

In [54]:
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
ratings.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [56]:
movies.head(4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [57]:
data = pd.merge(ratings, movies, on='movieId')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Sprawdź:
- Liczbę wszystkich ocen
- Liczbę filmów w bazie
- Liczbę użytkowników
- Średnią liczbę ocen na użytkownika
- Średnią liczbę ocen na film
- Rozkład ocen

In [71]:
#liczba wszystkich ocen
num_ratings= data['rating'].count()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
# Liczba filmów w bazie
num_movies = data['title'].nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
duplicated_titles = data[data.duplicated(subset=['title'], keep=False)]['title'].unique()
duplicated_titles

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', ...,
       'Blow-Out (La grande bouffe) (1973)', 'Misfits, The (1961)',
       'Cotton Club, The (1984)'], dtype=object)

In [70]:
# Liczbę użytkowników
num_users =data['userId'].nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
# Średnią liczbę ocen na użytkownika
mean = num_ratings/num_users
mean

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


165.30491803278687

In [74]:
# Średnia liczba ocen na film
mean_by_movie = num_ratings/num_movies
mean_by_movie

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


10.375141475460438

In [79]:
#najczęsciej oceniany film
ratings_count = data.groupby('title').size()
most_rated_movie = ratings_count.idxmax()

most_rated_movie

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Forrest Gump (1994)'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [91]:
movie_titles = dict(zip(movies['movieId'],movies['title']))
# movie_titles

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# C to średnia liczba ocen
# m reprezentuje średnią ocenę wszystkich filmów

In [96]:
c= data['rating'].count()/data['title'].count()
c

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1.0

In [97]:
#liczba filmów unikatowych/liczba ocen


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [108]:
from scipy.sparse import csr_matrix

def create_sparse(data):
  M = data['userId'].nunique()
  N = data['movieId'].nunique()
  user_mapper = dict(zip(np.unique(data["userId"]), list(range(M))))
  movie_mapper = dict(zip(np.unique(data["movieId"]), list(range(N))))
  user_inv_mapper = dict(zip(list(range(M)), np.unique(data["userId"])))
  movie_inv_mapper = dict(zip(list(range(N)), np.unique(data["movieId"])))
  user_index = [user_mapper[i] for i in data['userId']]
  item_index = [movie_mapper[i] for i in data['movieId']]
  X = csr_matrix((data["rating"], (user_index, item_index)), shape=(M,N))
  return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# create_sparse(data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
M = data['userId'].nunique()
N = data['movieId'].nunique()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
M = data['userId'].nunique()
N = data['movieId'].nunique()
user_mapper = dict(zip(np.unique(data["userId"]), list(range(M))))
movie_mapper = dict(zip(np.unique(data["movieId"]), list(range(N))))
user_inv_mapper = dict(zip(list(range(M)), np.unique(data["userId"])))
movie_inv_mapper = dict(zip(list(range(N)), np.unique(data["movieId"])))
user_index = [user_mapper[i] for i in data['userId']]
item_index = [movie_mapper[i] for i in data['movieId']]
X = csr_matrix((data["rating"], (user_index, item_index)), shape=(M,N))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz #scipy.sparse.csr_matrix.nnz no of stored values, including explicit 0
sparsity = n_ratings/n_total
sparsity_p = round(sparsity*100,2)
print(f"Matrix sparsity: {sparsity_p}%")
n_ratings_per_user = X.getnnz(axis=1)

Matrix sparsity: 1.7%


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [116]:
import scipy.sparse as sp

# Obliczenie liczby wszystkich możliwych ocen (łącznie z ocenami równymi zero)
n_total = X.shape[0] * X.shape[1]

# Obliczenie liczby ocenanych wartości (niezerowych) w macierzy rzadkiej
n_ratings = X.nnz

# Obliczenie rzadkości (sparsity) macierzy
sparsity = n_ratings / n_total
sparsity_p = round(sparsity * 100, 2)

print(f"Matrix sparsity: {sparsity_p}%")

# Obliczenie liczby ocen na użytkownika
n_ratings_per_user = X.getnnz(axis=1)


Matrix sparsity: 1.7%


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
